In [1]:
! pip3 install -q --upgrade --user google-cloud-aiplatform==1.60.0

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import vertexai

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}

print(PROJECT_ID)

vertexai.init(project=PROJECT_ID, location=LOCATION)

qwiklabs-gcp-01-ecd5ad280697


In [2]:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

In [11]:
def reverse(string):
    print("Calling reverse function")
    print(string)
    return string[::-1]

def remove_white_spaces(string):
    print("Calling remove_white_spaces function")
    return string.replace(" ","")


In [14]:
def handle_response(response):

    # If there is a function call then invoke it
    # Otherwise print the response.
    if response.candidates[0].function_calls:
        function_call = response.candidates[0].function_calls[0]
        print(function_call)

    else:
        print(response.text)
        return
    
    if function_call.name == "reverse":
        string_to_reverse = function_call.args["string"]
        print(string_to_reverse)
        reversed_string = reverse(string_to_reverse)
        res = chat.send_message(reversed_string)
        
        handle_response(res)
        # Extract the arguments to use in your function
        # Call your function
        # Send the result back to the chat session with the model
        # Recursive call
    
    elif function_call.name == "remove_white_spaces":
        string_to_clean = function_call.args["string"]
        cleaned_string = remove_white_spaces(string_to_clean)
        res = chat.send_message(cleaned_string)
        handle_response(res) 
    # the function_call requests your remove_white_spaces function
        # Extract the arguments to use in your function
        # Call your function
        # Send the result back to the chat session with the model
        # Make a recursive call of this handler function
    
    else:
        # You shouldn't end up here
        print(response.text)


In [15]:
reverse_func = FunctionDeclaration(
    name = "reverse",
    parameters={
        "type": "object",
        "properties": {
            "string": {
                "type": "string",
                "description": "string parameter",
            }
        },
    },
    description = "Reverse a string",
)

remove_white_spaces_func = FunctionDeclaration(
    name = "remove_white_spaces",
    parameters={
        "type": "object",
        "properties": {
            "string": {
                "type": "string",
                "description": "string parameter",
            }
        },
    },
    description = "Remove white spaces",
)

In [16]:
tool = Tool(
    function_declarations=[
        reverse_func,
        remove_white_spaces_func,
    ]
)

In [17]:
system_instruction = """
    - Fulfill the user's instructions.
    - If asked to reverse a string or remove whitespace, call the provided functions.
    - You may call one function after the other if needed.
    - Repeat the result to the user.
"""


model = GenerativeModel(
    model_name="gemini-1.5-pro-001",
    tools = [tool],
    generation_config=GenerationConfig(temperature=0),
    system_instruction = system_instruction
)

In [18]:
chat = model.start_chat()

In [19]:
response = chat.send_message("Tell me a joke.")
handle_response(response)

Why don't scientists trust atoms?

Because they make up everything! 



In [21]:
response = chat.send_message("Reverse the following string: Hello world")
handle_response(response)

name: "reverse"
args {
  fields {
    key: "string"
    value {
      string_value: "Hello world"
    }
  }
}

Hello world
Calling reverse function
Hello world
dlrow olleH



In [20]:
response = chat.send_message("remove white spaces in the following string: Hello world")
handle_response(response)

name: "remove_white_spaces"
args {
  fields {
    key: "string"
    value {
      string_value: "Hello world"
    }
  }
}

Calling remove_white_spaces function
Helloworld


In [24]:
response = chat.send_message("Remove the white spaces from the following word: hello @ world. com")
handle_response(response)

name: "remove_white_spaces"
args {
  fields {
    key: "string"
    value {
      string_value: "hello @ world. com"
    }
  }
}

Calling remove_white_spaces function
hello@world.com 



In [22]:
response = chat.send_message("Reverse the following string and remove spaces: I do not understand the new sentence.")
handle_response(response)   

name: "reverse"
args {
  fields {
    key: "string"
    value {
      string_value: "I do not understand the new sentence."
    }
  }
}

I do not understand the new sentence.
Calling reverse function
I do not understand the new sentence.
name: "remove_white_spaces"
args {
  fields {
    key: "string"
    value {
      string_value: ".ecnetnes wen eht dnatsrednu ton od I"
    }
  }
}

Calling remove_white_spaces function
.ecnetneswenehtdnatsrednutonodI



In [23]:
response = chat.send_message("Tell me a fact. Then remove the spaces in the fact text and then reverse the fact text.")
print(response.candidates[0].content.parts[0].text)
handle_response(response)  

The population of the earth is about 8 billion people. 


name: "remove_white_spaces"
args {
  fields {
    key: "string"
    value {
      string_value: "The population of the earth is about 8 billion people."
    }
  }
}

Calling remove_white_spaces function
name: "reverse"
args {
  fields {
    key: "string"
    value {
      string_value: "Thepopulationoftheearthisabout8billionpeople."
    }
  }
}

Thepopulationoftheearthisabout8billionpeople.
Calling reverse function
Thepopulationoftheearthisabout8billionpeople.
.elpoepnoillib8tuobasihtraeehtfonoitalupopehT 

